In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [17]:
start_date = '2022-10-01'
end_date = '2022-10-31'

In [18]:
taxi = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(taxi_lifetime_last_ride_date) 
    taxi_lifetime_last_ride_date,
    taxi_lifetime_first_ride_date,
    --MAX(taxi_lifetime_rides) 
    taxi_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (taxi_lifetime_rides > 0 or taxi_lifetime_rides is not null or taxi_lifetime_last_ride_date is not null)

    AND taxi_lifetime_rides BETWEEN 1 AND 4
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN taxi_lifetime_rides IS NULL OR DATE(month) = DATE(taxi_lifetime_first_ride_date) THEN 'LTR =0'
    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 AND taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN 'LTR ' || cast(taxi_lifetime_rides as varchar)
    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 THEN 'LTR 1-4'
    WHEN (taxi_lifetime_rides BETWEEN 5 AND 20) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 5-20'
    WHEN (taxi_lifetime_rides BETWEEN 21 AND 50) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 21-50'
    WHEN (taxi_lifetime_rides > 50) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 50+'
    WHEN taxi_lifetime_rides >= 5 THEN 'LTR >=5'
    ELSE 'LTR =0'
    END ltr_taxi,
    CASE 
    WHEN taxi_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(taxi_lifetime_first_ride_date) THEN 'LTR=0'

    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 and taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '0' DAY) THEN '1. 1_7_DAYS'
    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 and taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '1. 8_14_DAYS'
    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 and taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '1. 15_21_DAYS'
    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 and taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '1. 22_30_DAYS'
    

    WHEN taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '1. LAST_30_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '2. 31_45_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '3. 46_60_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '4. 61_90_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '5. 91_180_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '6. 181_365_DAYS'
    WHEN taxi_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '7. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_taxi
FROM base_month_customer
WHERE taxi_lifetime_rides BETWEEN 1 AND 4 AND taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name IN ('Link', 'Auto')
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_taxi,'NEW') ltr_taxi,
        COALESCE(b.recency_taxi, 'NEW') recency_taxi,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_taxi,
        case 
        when ltr_taxi in ('LTR 1', 'LTR 2', 'LTR 3', 'LTR 4') then 1
        when ltr_taxi = 'LTR 1-4' then 2
        when ltr_taxi = 'LTR 5-20' then 3
        when ltr_taxi = 'LTR 21-50' then 4
        when ltr_taxi = 'LTR 50+' then 5
        when ltr_taxi = 'LTR >=5' then 6
        when ltr_taxi = 'LTR =0' then 7
        when ltr_taxi = 'NEW' then 8
        end row_order,
        recency_taxi,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_taxi,
        a.recency_taxi,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_taxi,
        '' recency_taxi,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_taxi,recency_taxi,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3

"""

In [19]:
df_taxi = pd.read_sql(taxi, conn)
df_taxi

,month,ltr_taxi,recency_taxi,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,LTR 3,1. 15_21_DAYS,86112,0.57,49352,57.31,43779,50.84,33452,...,27.89,71.78,0.37,51145,0.23,2.13,200208.0,4.836553e+06,4.139477,3.914518
1,2022-10-01,LTR 4,1. 15_21_DAYS,65701,0.44,42495,64.68,38324,58.33,29776,...,32.94,72.69,0.33,47204,0.22,2.18,167141.0,4.420152e+06,3.781341,3.540823
2,2022-10-01,LTR 2,1. 15_21_DAYS,106615,0.71,59790,56.08,51813,48.60,38346,...,25.36,70.50,0.42,56126,0.26,2.08,254022.0,5.323448e+06,4.771757,4.525924
3,2022-10-01,LTR 1,1. 15_21_DAYS,156894,1.04,76412,48.70,62399,39.77,43919,...,18.63,66.54,0.45,58565,0.27,2.00,303082.0,5.630333e+06,5.383021,5.175139
4,2022-10-01,LTR 1,1. 1_7_DAYS,203754,1.36,133445,65.49,113619,55.76,85472,...,30.37,72.40,0.96,155763,0.71,2.52,750226.0,1.419241e+07,5.286109,4.816458
5,2022-10-01,LTR 4,1. 1_7_DAYS,128794,0.86,106860,82.97,101285,78.64,88826,...,58.44,84.74,1.16,234489,1.07,3.12,768662.0,2.103546e+07,3.654125,3.278030
6,2022-10-01,LTR 3,1. 1_7_DAYS,152346,1.01,120066,78.81,112829,74.06,97353,...,53.55,83.80,1.26,238627,1.09,2.93,851946.0,2.146083e+07,3.969772,3.570200
7,2022-10-01,LTR 2,1. 1_7_DAYS,175707,1.17,135046,76.86,123959,70.55,104164,...,48.52,81.85,1.32,227331,1.04,2.67,949857.0,2.062490e+07,4.605389,4.178299
8,2022-10-01,LTR 4,1. 22_30_DAYS,77107,0.51,45413,58.90,40514,52.54,30536,...,28.11,70.99,0.34,44402,0.20,2.05,158886.0,4.209582e+06,3.774389,3.578352
9,2022-10-01,LTR 3,1. 22_30_DAYS,105663,0.70,53576,50.70,47161,44.63,34828,...,22.96,69.65,0.37,49032,0.22,2.02,194226.0,4.672690e+06,4.156621,3.961209


In [20]:
df_taxi.to_csv("oct-taxi-ltr_1-4.csv", index = False)

In [16]:
df_taxi.to_csv("nov-taxi-ltr_1-4.csv", index = False)

## ltr 0

In [4]:
ltr_0 = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(taxi_lifetime_last_ride_date) 
    taxi_lifetime_last_ride_date,
    taxi_lifetime_first_ride_date,
    --MAX(taxi_lifetime_rides) 
    taxi_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (taxi_lifetime_rides > 0 or taxi_lifetime_rides is not null or taxi_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

ao_recent_ltr0 as
(
-- SELECT
--     customer_id,
--     max(day) ao_recency
-- FROM
    (
    SELECT 
        customerid customer_id,
        max(day) ao_recency
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)) - INTERVAL '365' DAY, '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        -- AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where taxi_lifetime_rides not between 1 and 10000)
        AND ao_sessions_unique_daily > 0
    GROUP BY 1
    )
-- GROUP BY 1
),

base_month_segment AS
(
SELECT * FROM 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    base_month_customer.customer_id,
    CASE
    WHEN taxi_lifetime_rides IS NULL OR DATE(month) = DATE(taxi_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 THEN '1-4'
    WHEN (taxi_lifetime_rides BETWEEN 5 AND 20) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '5-20'
    WHEN (taxi_lifetime_rides BETWEEN 21 AND 50) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '21-50'
    WHEN (taxi_lifetime_rides > 50) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '50+'
    WHEN taxi_lifetime_rides >= 5 THEN '>=5'
    ELSE 'LTR=0'
    END ltr_taxi,
    
    CASE 
    WHEN taxi_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(taxi_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN (taxi_lifetime_rides BETWEEN 1 AND 4) AND taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) THEN '1. LAST_7_DAYS'
    WHEN (taxi_lifetime_rides BETWEEN 1 AND 4) AND taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN (taxi_lifetime_rides BETWEEN 1 AND 4) AND taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_21_DAYS'
    WHEN (taxi_lifetime_rides BETWEEN 1 AND 4) AND taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '4. 22_30_DAYS'
    WHEN (taxi_lifetime_rides BETWEEN 1 AND 4) AND taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '5. 31_45_DAYS'
    WHEN (taxi_lifetime_rides BETWEEN 1 AND 4) AND taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '6. 46_60_DAYS'
    WHEN (taxi_lifetime_rides BETWEEN 1 AND 4) AND taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '7. 61_90_DAYS'    
    
    WHEN taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '11. LAST_30_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '12. 31_90_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '8. 91_180_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '9. 181_365_DAYS'
    WHEN taxi_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '10. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_taxi,
    
    CASE
    WHEN DATE(ao_recency) >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) THEN '1. LAST_7_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_30_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '4. 31_60_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '5. 61_90_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '6. 91_180_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '7. 181_365_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '730' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '366' DAY) THEN '8.1_2_YEARS'
    ELSE 'NO_AO'
    END ao_recency
FROM base_month_customer 
LEFT JOIN ao_recent_ltr0 b on base_month_customer.customer_id = b.customer_id
-- where taxi_lifetime_rides not between 1 and 10000
)
WHERE ltr_taxi = 'LTR=0' AND recency_taxi = 'LTR=0'
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name in ('Auto','Link')
    AND customerid IS NOT NULL
    AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where taxi_lifetime_rides not between 1 and 10000)
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
        AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where taxi_lifetime_rides not between 1 and 10000)
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_taxi,'NEW') ltr_taxi,
        COALESCE(b.recency_taxi, 'NEW') recency_taxi,
        ao_recency,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_taxi,
        case 
        when ltr_taxi = '1-4' then 1
        when ltr_taxi = '5-20' then 2
        when ltr_taxi = '21-50' then 3
        when ltr_taxi = '50+' then 4
        when ltr_taxi = '>=5' then 5
        when ltr_taxi = 'LTR=0' then 6
        when ltr_taxi = 'NEW' then 7
        end row_order,
        recency_taxi,
        ao_recency,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4,5
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_taxi,
        a.recency_taxi,
        ao_recency,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

-- b as 
-- (
-- SELECT
--         month,
--         8 row_order,
--         'TOTAL' ltr_auto,
--         '' recency_auto,
--         '' ao_recency,
--         SUM(existing_custr_count) existing_custr_count,
--         100.00 existing_custr_distr,
--         SUM(ao_users) ao_users,
--         COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
--         SUM(fe_users) fe_users,
--         COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
--         SUM(rr_users) rr_users,
--         COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
--         COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
--         SUM(net_users) net_users,
--         COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
--         COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
--         100.00 net_users_distr,
--         SUM(net_rides) net_rides,
--         100.00 net_rides_distr,
--         COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
--         SUM(discount) discount,
--         SUM(subtotal) subtotal
-- FROM a
-- GROUP BY 1
-- ),

final as 
(
select * from a
-- union
-- select * from b
)
select 
        month,
        ltr_taxi,recency_taxi,ao_recency,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,4

"""

In [5]:
df_ltr_0 = pd.read_sql(ltr_0, conn)
df_ltr_0

,month,ltr_taxi,recency_taxi,ao_recency,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-11-01,LTR=0,LTR=0,1. LAST_7_DAYS,361411,1.08,239165,66.18,189752,52.50,...,25.77,64.49,1.71,238119,1.12,2.56,1670073.0,2.107964e+07,7.922683,7.013607
1,2022-11-01,LTR=0,LTR=0,2. 8_14_DAYS,217587,0.65,90068,41.39,65112,29.92,...,12.06,59.96,0.48,52006,0.25,1.98,401191.0,4.996163e+06,8.029982,7.714321
2,2022-11-01,LTR=0,LTR=0,3. 15_30_DAYS,380687,1.14,111203,29.21,79491,20.88,...,8.19,60.21,0.57,59249,0.28,1.90,489433.0,5.621854e+06,8.705900,8.260612
3,2022-11-01,LTR=0,LTR=0,4. 31_60_DAYS,434351,1.30,80847,18.61,58378,13.44,...,5.32,61.12,0.42,42677,0.20,1.85,362595.0,3.977390e+06,9.116405,8.496263
4,2022-11-01,LTR=0,LTR=0,5. 61_90_DAYS,372598,1.12,44121,11.84,33276,8.93,...,3.59,62.01,0.25,25172,0.12,1.88,214600.0,2.363013e+06,9.081626,8.525346
5,2022-11-01,LTR=0,LTR=0,6. 91_180_DAYS,836834,2.51,58466,6.99,48647,5.81,...,2.42,63.89,0.37,38964,0.18,1.93,337643.0,3.479072e+06,9.704973,8.665512
6,2022-11-01,LTR=0,LTR=0,7. 181_365_DAYS,839065,2.51,35966,4.29,31688,3.78,...,1.68,66.66,0.26,28219,0.13,2.00,244060.0,2.425984e+06,10.060248,8.648783
7,2022-11-01,LTR=0,LTR=0,NO_AO,1665044,4.99,40198,2.41,35764,2.15,...,1.07,71.57,0.33,38247,0.18,2.14,322342.0,3.157554e+06,10.208598,8.427903
8,2022-11-01,NEW,NEW,None,0,0.00,9215535,0.00,8034839,0.00,...,0.00,80.10,95.62,20662183,97.53,3.96,63405006.0,1.631798e+09,3.885592,3.068650


In [6]:
df_ltr_0['existing_custr_count'].sum()

5107577

In [7]:
df_ltr_0.to_csv("nov-taxi-ltr 0.csv", index = False)